This contains part of the code for parsing through Java source code to obtain function names using regular expressions. Note that this is not used in the final version as the 'javalang' module in python can be used to convert the code to an AST directly. 

First, we note that all test functions have a '@Test' annotation before the function declaration. However, the issue is that there could be any number of other annotations between '@Test' and the declaration of the function itself.

In [ ]:
functions = dict()

# Open all of the test files 
for file in associatedFiles.keys(): 
    # Make sure that we have both the test file and the source 
    if (len(associatedFiles[file]) == 2): 
        # Open the test file 
        with open(associatedFiles[file][1], "r") as f: 
            content = f.readlines()
            for index, line in enumerate(content):
                res = re.match("[ ]*@Test", line)
                if res == None: 
                    continue 
                nextLine = content[index + 1]
                if re.match("[ ]*@Annotated", nextLine) != None:
                    index += 2
                else: 
                    index += 1
                
                # Check that it is not a comment 
                res = re.match("[ ]*//", content[index])
                if res is not None:
                    index += 1
                
                # Check for '@SuppressWarnings'
                res = re.match("[ ]*@SuppressWarnings", content[index])
                if res is not None: 
                    index += 1
                
                if file in functions: 
                    functions[file].append(content[index].strip())
                else: 
                    functions[file] = [content[index].strip()]

pprint(functions) 

In [ ]:
# Removed the prefix ('public void test_...'), 
# ignoring helper functions
actual_functions = dict()
for class_name in functions:
    functionsList = functions[class_name]
    for function in functionsList:
        # Get the name of the function 
        res = re.match("[ ]*[A-Za-z][A-Za-z]* [A-Za-z][A-Za-z]* ([A-Za-z][A-z_0-9]*)", function)
        function_name = res.groups()[0]

        # All functions start with 'test'
        if function_name[:4] == "test": 
            if class_name in actual_functions: 
                actual_functions[class_name].append(function_name[4:])
            else: 
                actual_functions[class_name] = [function_name[4:]]

pprint(actual_functions)

In [1]:
functionSet = dict()

# Open the real files 
for file in associatedFiles.keys(): 
    if len(associatedFiles[file]) == 2:
        # Open the file 
        with open(associatedFiles[file][0], "r") as f: 
            content = f.readlines() 
            for line in content:
                r = re.match("([ ]*)(public)(.*)", line)
                r1 = re.match("([ ]*)(protected)(.*)", line) 
                r2 = re.match("([ ]*)(private)(.*)", line)
                
                if r:
                    new_line = r.groups()[2]
                elif r1:
                    new_line = r1.groups()[2]
                elif r2:
                    new_line = r2.groups()[2]
                else:
                    continue 
                    
                if re.match("enum", new_line):
                    continue
                L = ["static", "void", "final", "long", "int", "boolean",
                     "Integer", "byte", "float", "double", "Date", "String"]
                
                for l in L:
                    r = re.match("[ ]*" + l + "(.*)", new_line)
                    if r:
                        new_line = r.groups()[0]
                r = re.match("(.*)\(.*\).*", new_line)              
                if re.match(".*=.*", new_line):
                    continue 
                    
                if (r):
                    if file in functionSet: 
                        functionSet[file].add(r.groups()[0])
                    else:
                        functionSet[file] = set() 
                        functionSet[file].add(r.groups()[0])
                    
pprint(functionSet)

NameError: name 'associatedFiles' is not defined

In [ ]:
for key in functionSet.keys(): 
    functions = functionSet[key]
    newFunctions = [] 
    for function in functions:
        function = function.strip()
#         print(function)
        while (True): 
            if function[:8] == 'abstract':
                function = function[8:].strip()
            elif function[:2] == "M " or function[:2] == "L " or function[:2] == "R " or function[:2] == "T ":
                function = function[2:].strip()
            elif function[:3] == "<T>": 
                function = function[3:].strip()
            elif function[:6] == "<L, R>":
                function = function[6:].strip()
            elif function[:9] == "<L, M, R>":
                function = function[9:].strip()
            else: 
                newFunctions.append(function)
                break 
    functionSet[key] = newFunctions
    pprint(newFunctions)
    

In [2]:
with open('functions.csv', 'w') as csvFile:
    writer = csv.writer(csvFile) 
    for key in functionSet.keys(): 
        row1 = [key]
        try: 
            row2 = list(functionSet[key])
            row3 = actual_functions[key]
            writer.writerow(row1)
            writer.writerow(row2)
            writer.writerow(row3)
        except: 
            print(key)

NameError: name 'csv' is not defined